<br/>

## <div style="padding:10px;background-color:#9d5a47;margin:10;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 1px 10px;overflow:hidden;font-weight:50;width:auto">Sony Churn Project - Data Preparation</div>

In [1]:
from IPython.display import Markdown, display

import pandas as pd
import numpy as np
np.random.seed(42)
rng = np.random.default_rng(42)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from pathlib import Path
data_path = Path('./data')


In [2]:
df = pd.read_csv(data_path / "Data_Science_Challenge.csv")
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


<br><br>

## Clean Data for modeling

In [3]:
# Change datatype of binary features to bool
for col in df.columns[df.columns.str.contains('plan')]:
    df[col] = df[col].map({'yes': True, 'no': False})
    

In [4]:
# check for string types
objs = df.dtypes[df.dtypes == 'object'].index.to_list()
objs

['state', 'phone number']

There is no guarantee that `state` will be encapsulated by `area code` and `phone number`

However, there will be some collinearity.  We choose to remove it

In [5]:
df = df.drop(['state', 'phone number'], axis=1)
df

,account length,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,128,415,False,True,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,False,True,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,False,False,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,408,True,False,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,415,True,False,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,False,True,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
3329,68,415,False,False,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
3330,28,510,False,False,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False
3331,184,510,True,False,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False


## Feature Engineering

Note there are some model specific changes we will have to make.  For instance, vmail messages will be skewed left when using a linear model like logistic regression - we would remove those without voicemail plan or we would change those without voicemail messages to the mean of those with.

In [6]:
# # add new column for df 'number vmail messages' with 'voice mail plan'
# df['num vmail messages with plan'] = df.apply(lambda x: x['number vmail messages'] if x['voice mail plan'] else 0, axis=1)
# df['num vmail messages without plan'] = df.apply(lambda x: x['number vmail messages'] if not x['voice mail plan'] else 0, axis=1)

# display((df['num vmail messages with plan'] == df['number vmail messages']).sum() == len(df), df)

## Save to a new dataset


In [8]:
# write to a parquet file
df.to_parquet(data_path / 'Data_Science_Challenge - prepped.parquet', index=False)